## Importing Libraries

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
import random

## Define Cnn architecture parameters ranges

In [ ]:
num_conv_layers_range = [1, 5]
num_pool_layers_range = [1, 5]
num_fc_layers_range = [1, 5]

## Particle parameters

In [ ]:
population_size = 50
max_generations = 50
c1 = 2.0  # cognitive parameter
c2 = 2.0  # social parameter
w = 0.7   # inertia weight

## Define Cnn architecture hyperparameters ranges

In [ ]:
num_filters_range = [1, 64]
padding_range = [0, 1]
stride_range = [1, 5]
filter_size_range = [1, 13]

## Load and Preprocess of MNIST dataset

In [ ]:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    input_shape = (28,28,1)
    num_classes = len(np.unique(y_train))
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

## Evaluate fitness for cnn_architecture

In [ ]:

def evaluate_fitness(cnn_architecture):

    num_conv_layers = int(cnn_architecture[0] * (num_conv_layers_range[1] - num_conv_layers_range[0]) + num_conv_layers_range[0])
    num_pool_layers = int(cnn_architecture[1] * (num_pool_layers_range[1] - num_pool_layers_range[0]) + num_pool_layers_range[0])
    num_fc_layers = int(cnn_architecture[2] * (num_fc_layers_range[1] - num_fc_layers_range[0]) + num_fc_layers_range[0])

    # Generate random hyperparameters for each layer
    filters = [random.randint(num_filters_range[0], num_filters_range[1]) for _ in range(num_conv_layers)]
    paddings = [random.randint(padding_range[0], padding_range[1]) for _ in range(num_conv_layers)]
    strides = [random.randint(stride_range[0], stride_range[1]) for _ in range(num_conv_layers)]
    filter_sizes = [random.randint(filter_size_range[0], filter_size_range[1]) for _ in range(num_conv_layers)]

    # Build CNN model
    model = Sequential()
    for i in range(num_conv_layers):
        model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28,28,1)))
        if paddings[i] > 0:
            model.add(Conv2D(filters[i], filter_sizes[i], padding='same', activation='relu'))
        if strides[i] > 1:
            model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    for _ in range(num_fc_layers):
        model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=64, epochs=1, validation_data=(x_test, y_test))

    # Evaluate the model
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)

    complexity = model.count_params()  # Total number of parameters in the model

    fitness = [accuracy, complexity]
    return fitness


## Create particle

In [ ]:
class Particle:
    def __init__(self, position):
        self.position = position
        self.velocity = np.zeros_like(position)
        self.best_position = np.copy(position)
        self.best_fitness = None

## MPSO algorithm

In [1]:
population = []
for _ in range(population_size):
    cnn_architecture = [
        random.random(),  # Number of convolutional layers
        random.random(),  # Number of pooling layers
        random.random()   # Number of fully connected layers
    ]
    population.append(Particle(cnn_architecture))

# Run the MPSO algorithm
for generation in range(1):
    for particle in population:
        # Evaluate fitness of the particle
        fitness = evaluate_fitness(particle.position)

        # Update personal best
        if particle.best_fitness is None or fitness[0] > particle.best_fitness[0]:
            particle.best_position = particle.position
            particle.best_fitness = fitness

    # Update global best
    population.sort(key=lambda p: p.best_fitness[0], reverse=True)
    global_best_fitness = population[0].best_fitness

    # Update velocity and position of each particle
    for particle in population:
        r1 = np.random.rand()
        r2 = np.random.rand()

        velocity = (
            w * particle.velocity +
            c1 * r1 * (particle.best_position - particle.position) +
            c2 * r2 * (population[0].best_position - particle.position)
        )
        particle.position += velocity
